In [2]:
import numpy as np
import pandas as pd

In [ ]:
# There are a number of schemes that have been developed to indicate the presence of missing data in a table or DataFrame. 
# Generally, they revolve around one of two strategies: using a mask that globally indicates missing values, or choosing a 
 # sentinel value that indicates a missing entry.

In [ ]:
# In the masking approach, the mask might be an entirely separate Boolean array, or it may involve appropriation of one bit 
# in the data representation to locally indicate the null status of a value.

In [ ]:
# In the sentinel approach, the sentinel value could be some data-specific convention, such as indicating a missing integer 
# value with -9999 or some rare bit pattern, or it could be a more global convention, such as indicating a missing 
# floating-point value with NaN (Not a Number), a special value which is part of the IEEE floating-point specification.

In [ ]:
# None of these approaches is without trade-offs: use of a separate mask array requires allocation of an additional Boolean 
# array, which adds overhead in both storage and computation. A sentinel value reduces the range of valid values that can be 
# represented, and may require extra (often non-optimized) logic in CPU and GPU arithmetic. Common special values like NaN are 
# not available for all data types.

In [ ]:
# Missing Data in Pandas

# The way in which Pandas handles missing values is constrained by its reliance on the NumPy package, which does not have a 
# built-in notion of NA values for non-floating-point data types.

In [ ]:
# Pandas could have followed R's lead in specifying bit patterns for each individual data type to indicate nullness, but this 
# approach turns out to be rather unwieldy. While R contains four basic data types, NumPy supports far more than this: for 
# example, while R has a single integer type, NumPy supports fourteen basic integer types once you account for available 
# precisions, signedness, and endianness of the encoding. Reserving a specific bit pattern in all available NumPy types would 
# lead to an unwieldy amount of overhead in special-casing various operations for various types, likely even requiring a new 
# fork of the NumPy package. 

In [ ]:
# With these constraints in mind, Pandas chose to use sentinels for missing data, and further chose to use two already-existing 
# Python null values: the special floating-point NaN value, and the Python None object. This choice has some side effects, as 
# we will see, but in practice ends up being a good compromise in most cases of interest.

In [ ]:
# None: Pythonic missing data

# The first sentinel value used by Pandas is None, a Python singleton object that is often used for missing data in Python code.
# Because it is a Python object, None cannot be used in any arbitrary NumPy/Pandas array, but only in arrays with data 
# type 'object' (i.e., arrays of Python objects):

In [6]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

In [ ]:
# This dtype=object means that the best common type representation NumPy could infer for the contents of the array is that they 
# are Python objects. While this kind of object array is useful for some purposes, any operations on the data will be done at 
# the Python level, with much more overhead than the typically fast operations seen for arrays with native types:

In [7]:
for dtype in ['object', 'int']:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype = object
64.3 ms ± 2.35 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype = int
2.6 ms ± 96.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)



In [8]:
# The use of Python objects in an array also means that if you perform aggregations like sum() or min() across an array with a 
# None value, you will generally get an error:
vals1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

In [ ]:
# NaN: Missing numerical data

# The other missing data representation, NaN (acronym for Not a Number), is different; it is a special floating-point value 
# recognized by all systems that use the standard IEEE floating-point representation:

In [11]:
vals2 = np.array([1, np.nan, 3, 4]) 
vals2.dtype

dtype('float64')

In [12]:
# Notice that NumPy chose a native floating-point type for this array: this means that unlike the object array from before, 
# this array supports fast operations pushed into compiled code. You should be aware that NaN is a bit like a data virus–it 
# infects any other object it touches. Regardless of the operation, the result of arithmetic with NaN will be another NaN:

1 + np.nan

nan

In [13]:
0 * np.nan

nan

In [14]:
# Note that this means that aggregates over the values are well defined (i.e., they don't result in an error) but not always 
# useful:
vals2.sum(), vals2.min(), vals2.max()

(nan, nan, nan)

In [15]:
# NumPy does provide some special aggregations that will ignore these missing values:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

In [ ]:
# Keep in mind that NaN is specifically a floating-point value; 
# there is no equivalent NaN value for integers, strings, or other types.

In [25]:
# NaN and None in Pandas

# NaN and None both have their place, and Pandas is built to handle the two of them nearly interchangeably, 
# converting between them where appropriate:
pd.Series([1, np.nan, 2, None])
# UPDATE: later versions of pandas can use pd.NA: 
# pd.Series([1, pd.NA, 2, None], dtype="Int64")

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

In [17]:
# For types that don't have an available sentinel value, Pandas automatically type-casts when NA values are present. 
# For example, if we set a value in an integer array to np.nan, it will automatically be upcast to a floating-point 
# type to accommodate the NA:

x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int32

In [19]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

In [ ]:
# Operating on Null Values
# As we have seen, Pandas treats None and NaN as essentially interchangeable for indicating missing or null values. 
# To facilitate this convention, there are several useful methods for detecting, removing, and replacing null values in 
# Pandas data structures. They are:

# isnull(): Generate a boolean mask indicating missing values
# notnull(): Opposite of isnull()
# dropna(): Return a filtered version of the data
# fillna(): Return a copy of the data with missing values filled or imputed

In [ ]:
# Detecting null values

In [ ]:
# isnull() and notnull(). Either one will return a Boolean mask over the data.

In [27]:
data = pd.Series([1, np.nan, 'hello', None, pd.NA])
data

0        1
1      NaN
2    hello
3     None
4     <NA>
dtype: object

In [28]:
data.isnull()

0    False
1     True
2    False
3     True
4     True
dtype: bool

In [30]:
# boolean masks can be used as a series or dataframe index:
data[data.notnull()]

0        1
2    hello
dtype: object

In [ ]:
# isnull() and notnull() produce similar values for dataframes

In [31]:
# Dropping null values

# In addition to the masking used before, there are the convenience methods, dropna() (which removes NA values) 
# and fillna() (which fills in NA values). For a Series, the result is straightforward:

data.dropna()

0        1
2    hello
dtype: object

In [33]:
# For a DataFrame, there are more options. Consider the following DataFrame:

df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [pd.NA, 4,      6]])
df

,0,1,2
0,1,NaN,2
1,2,3.0,5
2,<NA>,4.0,6


In [34]:
# We cannot drop single values from a DataFrame; we can only drop full rows or full columns. 
# Depending on the application, you might want one or the other, so dropna() gives a number of options for a DataFrame.

# By default, dropna() will drop all rows in which any null value is present:
df.dropna()

,0,1,2
1,2,3.0,5


In [35]:
# Alternatively, you can drop NA values along a different axis; axis=1 drops all columns containing a null value:
df.dropna(axis='columns')

,2
0,2
1,5
2,6


In [ ]:
# But this drops some good data as well; you might rather be interested in dropping rows or columns with all NA values, or a 
# majority of NA values. This can be specified through the how or thresh parameters, which allow fine control of the number of 
# nulls to allow through.

# The default is how='any', such that any row or column (depending on the axis keyword) containing a null value will be dropped. 
# You can also specify how='all', which will only drop rows/columns that are all null values:

In [36]:
df[3] = np.nan
df

,0,1,2,3
0,1,NaN,2,NaN
1,2,3.0,5,NaN
2,<NA>,4.0,6,NaN


In [37]:
df.dropna(axis='columns', how='all')

,0,1,2
0,1,NaN,2
1,2,3.0,5
2,<NA>,4.0,6


In [41]:
# For finer-grained control, the thresh parameter lets you specify a minimum number of non-null values for the row/column to 
# be kept:
# Here the first and last row have been dropped, because they contain only two non-null values.
df.dropna(axis='rows', thresh=3)

,0,1,2,3
1,2,3.0,5,NaN


In [ ]:
# Filling null values

In [ ]:
# You could do this in-place using the isnull() method as a mask, but because it is such a common operation Pandas provides 
# the fillna() method, which returns a copy of the array with the null values replaced.

In [42]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [43]:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [44]:
# We can specify a forward-fill to propagate the previous value forward:
# forward-fill
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [45]:
# Or we can specify a back-fill to propagate the next values backward:
# back-fill
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

In [46]:
# For DataFrames, the options are similar, but we can also specify an axis along which the fills take place:
df

,0,1,2,3
0,1,NaN,2,NaN
1,2,3.0,5,NaN
2,<NA>,4.0,6,NaN


In [49]:
df.fillna(method='ffill', axis=1)

,0,1,2,3
0,1,1,2,2
1,2,3,5,5
2,<NA>,4,6,6


In [ ]:
# Notice that if a previous value is not available during a forward fill, the NA value remains.